In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.document_loaders import UnstructuredFileLoader
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature = 0.1,)

loader = UnstructuredFileLoader("./document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"context": retriver, "question": RunnablePassthrough(), "history": load_memory} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    print(load_memory(""))
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [2]:
invoke_chain("Is Aaronson guilty")

[]
According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [3]:
invoke_chain("What message did he write int the table?")

[HumanMessage(content='Is Aaronson guilty'), AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')]
He wrote "2+2=5" in the dust on the table.


In [4]:
invoke_chain("Who is Julia?")

[HumanMessage(content='Is Aaronson guilty'), AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write int the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.')]
Julia is a character in the document who was involved with the protagonist in a forbidden relationship.
